<a href="https://colab.research.google.com/github/mikexie360/data-visualization-final-project/blob/main/data_visualization_final_project_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Collection from OpenDota API for Final Project
This google colab collects data from the top 5000 teams, team information such as hero picks, win rate and rating. And also their match history from patch 7.33 and up.

Running this google colab can take upwards of a few hours to collect all the data that you need.
You also might need an OpenDota API Key.

In [24]:
## imports
import requests
import pandas as pd
from datetime import datetime, timezone
import time

from google.colab import userdata


In [37]:
# ==== CONFIG ====
OPENDOTA_API_KEY = userdata.get('opendota_api_key')
PATCH_MIN = 7.33        # 7.33 and up
LIMIT = 5000            # keep this moderate; huge pages + JSON can 400
START_OFFSET = 0
MAX_PAGES = None        # None = keep going until empty; or an int
TIMEOUT = 60

API_BASE = "https://api.opendota.com/api"
SLEEP_SEC = 0.25           # gentle rate-limit
MAX_TEAMS = None           # set an int to cap for testing; or None for all

In [23]:
## teams_df

## top 5000 teams

all_batches = []
for page in range(5):  # pages 0..4
    params = {"api_key": OPENDOTA_API_KEY, "page": page}
    resp = requests.get("https://api.opendota.com/api/teams", params=params, timeout=30)
    resp.raise_for_status()
    batch = resp.json()
    if not batch:
        break
    all_batches.append(pd.DataFrame(batch))

# combine pages
teams_df = pd.concat(all_batches, ignore_index=True)

# add readable timestamp (UTC ISO-8601)
teams_df["last_match_time_iso"] = pd.to_datetime(
    teams_df["last_match_time"], unit="s", utc=True, errors="coerce"
).dt.strftime("%Y-%m-%dT%H:%M:%SZ")

# select/ordering columns
cols = [
    "team_id", "name", "tag", "rating", "wins", "losses",
    "last_match_time", "last_match_time_iso", "logo_url"
]
teams_df = teams_df.reindex(columns=cols)

# write once
teams_df.to_csv("teams.csv", index=False, encoding="utf-8")
print(teams_df.head())
print(f"Number of rows: {len(teams_df)}")

   team_id          name         tag   rating  wins  losses  last_match_time  \
0  7119388   Team Spirit     TSpirit  1578.63   768     512       1752941276   
1  9572001    PARIVISION  PARIVISION  1559.36   179      91       1753691348   
2  8255888  BetBoom Team     BetBoom  1547.12   502     405       1753694464   
3  7412785  CyberBonch-1          CB  1520.12   267       1       1639928575   
4  8605863        Cloud9          C9  1497.49   326     243       1726305637   

    last_match_time_iso                                           logo_url  
0  2025-07-19T16:07:56Z  https://cdn.steamusercontent.com/ugc/183917912...  
1  2025-07-28T08:29:08Z  https://cdn.steamusercontent.com/ugc/247650869...  
2  2025-07-28T09:21:04Z  https://cdn.steamusercontent.com/ugc/999542643...  
3  2021-12-19T15:42:55Z  https://cdn.steamusercontent.com/ugc/184253787...  
4  2024-09-14T09:20:37Z  https://cdn.steamusercontent.com/ugc/239994188...  
Number of rows: 5000


In [11]:
## heroes_df

### All heroes in the game

heroes_url = "https://api.opendota.com/api/heroes"
resp = requests.get(heroes_url)
resp.raise_for_status()
data = resp.json()  # <- list of dicts

# optional: turn the roles list into a comma-separated string
for row in data:
    if isinstance(row.get("roles"), list):
        row["roles"] = ",".join(row["roles"])

# pick a column order (optional)
cols = ["id", "name", "localized_name", "primary_attr", "attack_type", "roles", "legs"]
heroes_df = pd.DataFrame(data)[cols]

heroes_df.to_csv("heroes.csv", index=False, encoding="utf-8")
print(heroes_df.head)


<bound method NDFrame.head of       id                          name  localized_name primary_attr  \
0      1        npc_dota_hero_antimage       Anti-Mage          agi   
1      2             npc_dota_hero_axe             Axe          str   
2      3            npc_dota_hero_bane            Bane          all   
3      4     npc_dota_hero_bloodseeker     Bloodseeker          agi   
4      5  npc_dota_hero_crystal_maiden  Crystal Maiden          int   
..   ...                           ...             ...          ...   
121  135     npc_dota_hero_dawnbreaker     Dawnbreaker          str   
122  136           npc_dota_hero_marci           Marci          all   
123  137    npc_dota_hero_primal_beast    Primal Beast          str   
124  138          npc_dota_hero_muerta          Muerta          int   
125  145             npc_dota_hero_kez             Kez          agi   

    attack_type                                    roles  legs  
0         Melee                       Carry,Escape,N

In [32]:
## matches_teams_picks_bans

API = "https://api.opendota.com/api/explorer"

# output filenames
MATCH_TEAMS_CSV = "match_teams_733_plus.csv"
PICKS_BANS_LONG_CSV = "match_picks_bans_long_733_plus.csv"
MATCHES_TEAMS_PB_WIDE_CSV = "matches_teams_picks_bans_733_plus.csv"
# ===================================================

def run_explorer(limit: int, offset: int) -> pd.DataFrame:
    sql = f"""
        SELECT
          m.match_id,
          m.start_time,
          m.radiant_team_id,
          m.dire_team_id,
          m.radiant_win,
          m.duration,
          m.radiant_score,
          m.dire_score,
          m.picks_bans,
          CASE WHEN m.radiant_win THEN m.radiant_team_id ELSE m.dire_team_id END AS winner_team_id,
          m.leagueid,
          l.name AS league_name,
          mp.patch
        FROM matches m
        JOIN match_patch mp USING (match_id)
        LEFT JOIN leagues l ON l.leagueid = m.leagueid
        WHERE m.radiant_team_id IS NOT NULL
          AND m.dire_team_id IS NOT NULL
          AND m.radiant_team_id > 0
          AND m.dire_team_id > 0
          AND NULLIF(regexp_replace(mp.patch, '[^0-9\\.]', '', 'g'), '') IS NOT NULL
          AND CAST(regexp_replace(mp.patch, '[^0-9\\.]', '', 'g') AS numeric) >= {PATCH_MIN}
        ORDER BY m.match_id
        LIMIT {limit} OFFSET {offset}
    """
    params = {"sql": sql}
    if OPENDOTA_API_KEY:
        params["api_key"] = OPENDOTA_API_KEY
    r = requests.get(API, params=params, timeout=TIMEOUT)
    r.raise_for_status()
    return pd.DataFrame(r.json().get("rows", []))

all_match_pages = []
all_pb_rows = []
page = 0
offset = START_OFFSET
prev_sample = None

while True:
    page += 1

    if prev_sample is not None:
        print(f"[Page {page}] Sample of previous page (first 5 rows):")
        print(prev_sample.to_string(index=False))
        print("-" * 80)

    print(f"Fetching page {page} (offset={offset}, limit={LIMIT}) ...")
    df = run_explorer(LIMIT, offset)
    n = len(df)
    print(f"Fetched {n} rows.")
    if n == 0:
        print("No more rows. Stopping.")
        break

    # Preview for next loop
    show_cols = [c for c in ["match_id", "radiant_team_id", "dire_team_id", "league_name", "patch"] if c in df.columns]
    prev_sample = df[show_cols].head(5).copy()

    # Accumulate matches
    all_match_pages.append(df)

    # Build a long-form picks/bans while we have the row context (to resolve team_id)
    if "picks_bans" in df.columns:
        for _, row in df.iterrows():
            match_id = row["match_id"]
            r_tid = row["radiant_team_id"]
            d_tid = row["dire_team_id"]
            pbs = row["picks_bans"]
            if isinstance(pbs, list):
                for pb in pbs:
                    side = pb.get("team")  # 0=radiant, 1=dire
                    team_id = r_tid if side == 0 else d_tid if side == 1 else None
                    all_pb_rows.append({
                        "match_id": match_id,
                        "order": pb.get("order"),
                        "is_pick": pb.get("is_pick"),
                        "hero_id": pb.get("hero_id"),
                        "team_side": side,
                        "team_id": team_id,
                    })

    offset += LIMIT
    if MAX_PAGES is not None and page >= MAX_PAGES:
        print("Reached MAX_PAGES cap. Stopping.")
        break

# ===== Build match_teams_df =====
match_teams_df = pd.concat(all_match_pages, ignore_index=True) if all_match_pages else pd.DataFrame()

# Add ISO timestamp
if "start_time" in match_teams_df.columns:
    match_teams_df["start_time_iso"] = pd.to_datetime(
        match_teams_df["start_time"], unit="s", utc=True, errors="coerce"
    ).dt.strftime("%Y-%m-%dT%H:%M:%SZ")

# Column order
preferred_cols = [
    "match_id",
    "radiant_team_id", "dire_team_id",
    "start_time", "start_time_iso",
    "radiant_win", "winner_team_id",
    "duration", "radiant_score", "dire_score",
    "leagueid", "league_name", "patch", "picks_bans"
]
match_teams_df = match_teams_df[[c for c in preferred_cols if c in match_teams_df.columns] +
                                [c for c in match_teams_df.columns if c not in preferred_cols]]

# ===== Build long-form picks/bans df (already team_id-resolved) =====
match_picks_bans_df = pd.DataFrame(all_pb_rows).sort_values(["match_id", "order"]).reset_index(drop=True)

# ===== Build WIDE matches_teams_picks_bans =====
# For each match row, create: isPick{n}, hero_id{n}, team_id{n}, order{n}
def flatten_picks_bans_row(row) -> dict:
    base = {k: row[k] for k in row.index if k != "picks_bans"}  # copy all except the JSON
    pbs = row.get("picks_bans")
    if isinstance(pbs, list):
        # ensure ordered by "order"
        pbs_sorted = sorted(pbs, key=lambda x: x.get("order", 0))
        for pb in pbs_sorted:
            k = int(pb.get("order", 0)) + 1  # 1-based suffix
            side = pb.get("team")  # 0=radiant, 1=dire
            team_id = row["radiant_team_id"] if side == 0 else row["dire_team_id"] if side == 1 else None
            base[f"isPick{k}"] = bool(pb.get("is_pick"))
            base[f"hero_id{k}"] = pb.get("hero_id")
            base[f"team_id{k}"] = team_id
            base[f"order{k}"] = pb.get("order")
    return base

if not match_teams_df.empty:
    wide_records = [flatten_picks_bans_row(r) for _, r in match_teams_df.iterrows()]
    matches_teams_picks_bans = pd.DataFrame(wide_records)
else:
    matches_teams_picks_bans = pd.DataFrame()

# Drop the raw JSON column from match_teams_df before saving (keeps file tidy)
if "picks_bans" in match_teams_df.columns:
    match_teams_df_nojson = match_teams_df.drop(columns=["picks_bans"])
else:
    match_teams_df_nojson = match_teams_df.copy()

# ===== SAVE CSVs =====
match_teams_df_nojson.to_csv(MATCH_TEAMS_CSV, index=False, encoding="utf-8")
match_picks_bans_df.to_csv(PICKS_BANS_LONG_CSV, index=False, encoding="utf-8")
matches_teams_picks_bans.to_csv(MATCHES_TEAMS_PB_WIDE_CSV, index=False, encoding="utf-8")

print("\nDone!")
print("Saved:", MATCH_TEAMS_CSV)
print("Saved:", PICKS_BANS_LONG_CSV)
print("Saved:", MATCHES_TEAMS_PB_WIDE_CSV)

print("\nShapes:")
print("match_teams_df_nojson:", match_teams_df_nojson.shape)
print("match_picks_bans_df:", match_picks_bans_df.shape)
print("matches_teams_picks_bans:", matches_teams_picks_bans.shape)

print("\nPreview matches_teams_picks_bans:")
print(matches_teams_picks_bans.head().to_string(index=False))

Fetching page 1 (offset=0, limit=5000) ...
Fetched 5000 rows.
[Page 2] Sample of previous page (first 5 rows):
  match_id  radiant_team_id  dire_team_id                                                         league_name patch
7116526799          8680612       8971308 DPC 2023 SA Spring Tour Division II – presented by ESB Liga Esports  7.33
7116656225          8893835       8864178                                                     Ancients League  7.33
7116662198          8629315       8629318                                                      Destiny league  7.33
7116708252          8864178       8893835                                                     Ancients League  7.33
7116718813          8863825       8736661          DPC 2023 SEA Spring Tour Division II - presented by Epulze  7.33
--------------------------------------------------------------------------------
Fetching page 2 (offset=5000, limit=5000) ...
Fetched 5000 rows.
[Page 3] Sample of previous page (first 5 rows)

In [38]:
## teams_heros_df

OUT_CSV = "team_hero_df.csv"

# --- Helper to fetch a team's hero stats ---
def fetch_team_heroes(team_id: int, api_key: str | None = None, timeout: int = 30) -> pd.DataFrame:
    url = f"{API_BASE}/teams/{team_id}/heroes"
    params = {"api_key": api_key} if api_key else None
    r = requests.get(url, params=params, timeout=timeout)
    r.raise_for_status()
    data = r.json()  # list[dict]
    df = pd.DataFrame(data)
    if df.empty:
        return pd.DataFrame(columns=["team_id", "hero_id"])
    df["team_id"] = team_id
    return df

# --- Optional: hero lookup (id -> names) ---
def get_heroes_lookup() -> pd.DataFrame:
    # Use existing heroes_df if present; else fetch once.
    if "heroes_df" in globals() and isinstance(heroes_df, pd.DataFrame) and "id" in heroes_df.columns:
        h = heroes_df.copy()
    else:
        r = requests.get(f"{API_BASE}/heroes", timeout=30)
        r.raise_for_status()
        h = pd.DataFrame(r.json())
    # Keep common columns; rename id -> hero_id for merge
    keep = [c for c in ["id", "name", "localized_name", "primary_attr", "attack_type"] if c in h.columns]
    h = h[keep].rename(columns={"id": "hero_id"})
    return h

# --- Collect team->hero stats for all teams in teams_df ---
team_ids = teams_df["team_id"].dropna().astype(int).unique()
if MAX_TEAMS is not None:
    team_ids = team_ids[:MAX_TEAMS]

all_frames = []
for i, tid in enumerate(team_ids, start=1):
    try:
        if i == 1:
            print(f"Starting team hero fetch for {len(team_ids)} teams...")
        if i % 50 == 1 or i == len(team_ids):
            print(f"[{i}/{len(team_ids)}] team_id={tid}")
        df = fetch_team_heroes(tid, api_key=OPENDOTA_API_KEY)
        all_frames.append(df)
    except Exception as e:
        print(f"Failed team_id={tid}: {e}")
    time.sleep(SLEEP_SEC)

team_hero_df = pd.concat(all_frames, ignore_index=True) if all_frames else pd.DataFrame()

# --- Add winrate if wins/games columns exist (naming can vary) ---
wins_col = "wins" if "wins" in team_hero_df.columns else ("win" if "win" in team_hero_df.columns else None)
games_col = "games_played" if "games_played" in team_hero_df.columns else ("games" if "games" in team_hero_df.columns else None)
if wins_col and games_col:
    team_hero_df["winrate"] = (team_hero_df[wins_col] / team_hero_df[games_col]).round(4)

# --- Attach hero names (id -> names) ---
try:
    heroes_lookup = get_heroes_lookup()
    team_hero_df = team_hero_df.merge(heroes_lookup, on="hero_id", how="left")
except Exception as e:
    print("Skipping hero name merge:", e)

# --- Nice column order (best-effort; keeps extras at the end) ---
preferred = ["team_id", "hero_id", "localized_name", "name", "primary_attr", "attack_type",
             "games_played", "games", "wins", "win", "winrate"]
ordered = [c for c in preferred if c in team_hero_df.columns]
rest = [c for c in team_hero_df.columns if c not in ordered]
team_hero_df = team_hero_df[ordered + rest]

# --- Save & peek ---
team_hero_df.to_csv(OUT_CSV, index=False, encoding="utf-8")
print("Saved:", OUT_CSV)
print("team_hero_df shape:", team_hero_df.shape)
print(team_hero_df.head().to_string(index=False))


Starting team hero fetch for 5000 teams...
[1/5000] team_id=7119388
[51/5000] team_id=9187066
[101/5000] team_id=9303484
[151/5000] team_id=8214850
[201/5000] team_id=8998465
[251/5000] team_id=8936568
[301/5000] team_id=1848465
[351/5000] team_id=9216247
[401/5000] team_id=3018001
[451/5000] team_id=2850822
[501/5000] team_id=8495691
[551/5000] team_id=1665758
[601/5000] team_id=8713367
[651/5000] team_id=7716098
[701/5000] team_id=7019918
[751/5000] team_id=2024718
[801/5000] team_id=8193371
[851/5000] team_id=7422393
[901/5000] team_id=8526136
[951/5000] team_id=2025160
[1001/5000] team_id=5006832
[1051/5000] team_id=6435695
[1101/5000] team_id=9683757
[1151/5000] team_id=9440022
[1201/5000] team_id=9466553
[1251/5000] team_id=7510730
[1301/5000] team_id=7578716
[1351/5000] team_id=8604954
[1401/5000] team_id=8999483
[1451/5000] team_id=9373820
[1501/5000] team_id=9530233
[1551/5000] team_id=2496863
[1601/5000] team_id=2913028
[1651/5000] team_id=9126284
[1701/5000] team_id=1709168


In [39]:
## data frames we have so far

## teams_df
print(teams_df.head)
print("number of rows " + str(len(teams_df)))
## heroes_df
print(heroes_df.head)
print("number of rows " + str(len(heroes_df)))
## matches_teams_picks_bans
print(matches_teams_picks_bans.head)
print("number of rows " + str(len(matches_teams_picks_bans)))
# match_teams_df_nojson
print(match_teams_df_nojson.head)
print("number of rows " + str(len(match_teams_df_nojson)))
# match_picks_bans_df
print(match_picks_bans_df.head)
print("number of rows " + str(len(match_picks_bans_df)))
## team_hero information
print(team_hero_df.head)
print("number of rows " + str(len(team_hero_df)))

<bound method NDFrame.head of       team_id                  name         tag   rating  wins  losses  \
0     7119388           Team Spirit     TSpirit  1578.63   768     512   
1     9572001            PARIVISION  PARIVISION  1559.36   179      91   
2     8255888          BetBoom Team     BetBoom  1547.12   502     405   
3     7412785          CyberBonch-1          CB  1520.12   267       1   
4     8605863                Cloud9          C9  1497.49   326     243   
...       ...                   ...         ...      ...   ...     ...   
4995   470237  High Skill Dickheads         HSD  1015.33     1       0   
4996  5635522            truestrike  Truestrike  1015.33     3       2   
4997  3703112             Coca Cola              1015.32     1       0   
4998  3326101               Too EzZ       !TE!.  1015.32     1       0   
4999  7358791               SVALINN       SVL |  1015.32     1       0   

      last_match_time   last_match_time_iso  \
0          1752941276  2025-07-19T

In [33]:
## that is all for data collection for now